Changes made by @fhalamos from https://github.com/jmather625/predicting-poverty-replication

Make sure you download the 2016 Household LSMS survey data for Malawi, Ethiopia, and Nigeria from https://microdata.worldbank.org/index.php/catalog/lsms and put it in `../data/countries/`. Malawi's data should be named `malawi_2016/LSMS`, Ethiopia's should be named `ethiopia_2015/LSMS`, and Nigeria's should be named `nigeria_2015/LSMS`. Nightlights data should be downloaded from https://ngdc.noaa.gov/eog/viirs/download_dnb_composites.html using the annual composite from 2015 in tile 2 and tile 5.

In [1]:
! pip install geoio
! pip install utils

In [2]:
import pandas as pd
import numpy as np
import os
import geoio

In [3]:
from google.colab import files, drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [30]:
BASE_DIR = '/content/drive/MyDrive/Data Science GRDS [Felipe]/Poverty Measurement'

/content


In [31]:

NIGHTLIGHTS_DIRS = [os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_00N060W.tif'),
                    os.path.join(BASE_DIR, 'data/nightlights/viirs_2015_75N060W.tif')]

COUNTRIES_DIR = os.path.join(BASE_DIR, 'data', 'countries')

In [28]:
import sys
sys.path.append(BASE_DIR)
from utils import create_space

ImportError: ignored

In [63]:
#Malawi parameters
mw_lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
mw_consumption_file = os.path.join('consumption_aggregate','ihs4 consumption aggregate.csv')
mw_consumption_ph_col = 'rexpagg' # per household
mw_hhsize_col = 'hhsize' # people in household

mw_geovariables_file = os.path.join('household_geovariables','householdgeovariablesihs4.csv')
mw_lat_col = 'lat_modified'
mw_lon_col = 'lon_modified'

# purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
mw_ppp = 215.182

mw_hh_id_col = 'case_id'
mw_country_abbreviation = 'mw'

def process_country_data(country_abbreviation, lsms_dir, consumption_file, hhsize_col, geovariables_file, lat_col, lon_col, ppp, hh_id_col, consumption_ph_col=None, consumption_pc_col=None):
    '''
    consumption_ph_col # per household
    consumption_pc_col # per capita
    '''
    #Check files exist
    for file in [consumption_file, geovariables_file]:       
        file_path = os.path.join(lsms_dir, file)
        assert os.path.isfile(file_path), print(f'Could not find {file_path}')
    
    #Load consumption df    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    
    #Create consumption per household column
    if consumption_ph_col:
      df['cons_ph'] = df[consumption_ph_col]
    #If consumption was reported per capita, multiply by hh size to get per household
    elif consumption_pc_col:
      df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]

    #Create people per household column
    df['pph'] = df[hhsize_col]

    # Create consumption per household adjusted pp
    df['cons_ph'] = df['cons_ph'] / ppp / 365

    #Filter df to only selected columns
    df = df[[hh_id_col, 'cons_ph', 'pph']]

    #Load geovariables df
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[[hh_id_col, lat_col, lon_col]]

    #Rename lat lon columns after cluster
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)

    #Merge geodata with consumption data on hh identifier
    df_combined = pd.merge(df, df_cords, on=hh_id_col)

    #Drop repeted identifier
    df_combined.drop([hh_id_col], axis=1, inplace=True)

    #Drop na values
    df_combined.dropna(inplace=True)

    #Aggregate consumption at the cluster level
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()

    #Compute consumption per capita
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = country_abbreviation
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]




#Load malawi data
df_mw = process_country_data(country_abbreviation = mw_country_abbreviation,
                     lsms_dir = mw_lsms_dir,
                     consumption_file = mw_consumption_file,
                     hhsize_col = mw_hhsize_col,
                     geovariables_file = mw_geovariables_file,
                     lat_col = mw_lat_col,
                     lon_col = mw_lon_col,
                     ppp = mw_ppp,
                     hh_id_col = mw_hh_id_col,
                     consumption_ph_col=mw_consumption_ph_col)
df_mw.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,country,cluster_lat,cluster_lon,cons_pc
0,mw,-17.095150,35.217213,1.423239
1,mw,-17.092351,35.114643,1.266204
2,mw,-17.016698,35.079629,1.566870
3,mw,-16.977243,35.205706,1.669245
4,mw,-16.956385,35.168967,1.089891


In [42]:
'''
The goal of each of these functions is to output a dataframe with the following columns:
country, cluster_lat, cluster_lon, cons_pc

Each row should represent one cluster by combining the household data
'''
def process_malawi():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'malawi_2016', 'LSMS')
    consumption_file = os.path.join('consumption_aggregate','ihs4 consumption aggregate.csv')
    consumption_ph_col = 'rexpagg' # per household
    hhsize_col = 'hhsize' # people in household

    geovariables_file = os.path.join('household_geovariables','householdgeovariablesihs4.csv')
    lat_col = 'lat_modified'
    lon_col = 'lon_modified'

   
    #Check files exist
    for file in [consumption_file, geovariables_file]:       
        file_path = os.path.join(lsms_dir, file)
        assert os.path.isfile(file_path), print(f'Could not find {file_path}')
    

    #Load consumption df    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    
    #Create cons_ph and pph columns
    df['cons_ph'] = df[consumption_ph_col]
    df['pph'] = df[hhsize_col]

    # Save consumption per household adjusted pp
    # purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
    ppp = 215.182
    df['cons_ph'] = df['cons_ph'] / ppp / 365

    #Filter df to only selected columns
    df = df[['case_id', 'cons_ph', 'pph']]

    #Load geovariables df
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['case_id', 'HHID', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)

    #Merge geodata with consumption data
    df_combined = pd.merge(df, df_cords, on='case_id')

    df_combined.drop(['case_id', 'HHID'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values

    #Aggregate consumption at the cluster level
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()

    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'mw'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [44]:
df_mw = process_malawi()
df_mw.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,country,cluster_lat,cluster_lon,cons_pc
0,mw,-17.095150,35.217213,1.423239
1,mw,-17.092351,35.114643,1.266204
2,mw,-17.016698,35.079629,1.566870
3,mw,-16.977243,35.205706,1.669245
4,mw,-16.956385,35.168967,1.089891


In [65]:
#Load ethipioa data

#Malawi parameters
eth_lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
eth_consumption_file = 'Consumption Aggregate/cons_agg_w3.csv'
eth_consumption_pc_col = 'total_cons_ann'
eth_hhsize_col = 'hh_size' # people in household

eth_geovariables_file = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
eth_lat_col = 'lat_dd_mod'
eth_lon_col = 'lon_dd_mod'


# purchasing power parity for malawi in 2016 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=MW)
eth_ppp = 7.882

eth_hh_id_col = 'household_id2'
eth_country_abbreviation = 'eth'

df_eth = process_country_data(country_abbreviation = eth_country_abbreviation,
                     lsms_dir = eth_lsms_dir,
                     consumption_file = eth_consumption_file,
                     hhsize_col = eth_hhsize_col,
                     geovariables_file = eth_geovariables_file,
                     lat_col = eth_lat_col,
                     lon_col = eth_lon_col,
                     ppp = eth_ppp,
                     hh_id_col = eth_hh_id_col,
                     consumption_pc_col=eth_consumption_pc_col)
df_eth.head()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



,country,cluster_lat,cluster_lon,cons_pc
0,eth,3.455701,39.515994,14.854634
1,eth,3.549937,39.184234,14.312022
2,eth,3.864243,39.101366,12.470145
3,eth,3.982931,38.491368,8.336539
4,eth,4.048194,41.930928,4.762526


In [53]:
def process_ethiopia():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'LSMS')
    consumption_file = 'Consumption Aggregate/cons_agg_w3.csv'
    consumption_pc_col = 'total_cons_ann' # per capita
    hhsize_col = 'hh_size' # people in household

    geovariables_file = 'Geovariables/ETH_HouseholdGeovars_y3.csv'
    lat_col = 'lat_dd_mod'
    lon_col = 'lon_dd_mod'

    # purchasing power parity for ethiopia in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=ET)
    ppp = 7.882
    
    #Check files exist
    for file in [consumption_file, geovariables_file]:
        file_path = os.path.join(lsms_dir, file)
        assert os.path.isfile(file_path), print(f'Could not find {file_path}')
    
    #Load consumption data and choose desired data
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['household_id2', 'cons_ph', 'pph']]

    #Load geovariables df
    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['household_id2', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    
    #Merge consumption and geovariables
    df_combined = pd.merge(df, df_cords, on='household_id2')
    df_combined.drop(['household_id2'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'eth'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [56]:
df_eth = process_ethiopia()

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [57]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc
0,eth,3.455701,39.515994,14.854634
1,eth,3.549937,39.184234,14.312022
2,eth,3.864243,39.101366,12.470145
3,eth,3.982931,38.491368,8.336539
4,eth,4.048194,41.930928,4.762526


In [37]:



def process_nigeria():
    lsms_dir = os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'LSMS')
    consumption_file = 'cons_agg_wave3_visit1.csv'
    consumption_pc_col = 'totcons' # per capita
    hhsize_col = 'hhsize' # people in household

    geovariables_file = 'nga_householdgeovars_y3.csv'
    lat_col = 'LAT_DD_MOD'
    lon_col = 'LON_DD_MOD'

    # purchasing power parity for nigeria in 2015 (https://data.worldbank.org/indicator/PA.NUS.PRVT.PP?locations=NG)
    ppp = 95.255
    
    for file in [consumption_file, geovariables_file]:
        assert os.path.isfile(os.path.join(lsms_dir, file)), print(f'Could not find {file}')
    
    df = pd.read_csv(os.path.join(lsms_dir, consumption_file))
    df['cons_ph'] = df[consumption_pc_col] * df[hhsize_col]
    df['pph'] = df[hhsize_col]
    df['cons_ph'] = df['cons_ph'] / ppp / 365
    df = df[['hhid', 'cons_ph', 'pph']]

    df_geo = pd.read_csv(os.path.join(lsms_dir, geovariables_file))
    df_cords = df_geo[['hhid', lat_col, lon_col]]
    df_cords.rename(columns={lat_col: 'cluster_lat', lon_col: 'cluster_lon'}, inplace=True)
    df_combined = pd.merge(df, df_cords, on='hhid')
    df_combined.drop(['hhid'], axis=1, inplace=True)
    df_combined.dropna(inplace=True) # can't use na values
    
    df_clusters = df_combined.groupby(['cluster_lat', 'cluster_lon']).sum().reset_index()
    df_clusters['cons_pc'] = df_clusters['cons_ph'] / df_clusters['pph'] # divides total cluster income by people
    df_clusters['country'] = 'ng'
    return df_clusters[['country', 'cluster_lat', 'cluster_lon', 'cons_pc']]

In [ ]:
df_ng = process_nigeria()

In [ ]:
df_mw.shape, df_eth.shape, df_ng.shape

((780, 4), (523, 4), (664, 4))

In [ ]:
tifs = [geoio.GeoImage(ndir) for ndir in NIGHTLIGHTS_DIRS]

In [ ]:
# loading both of these into memory requires A LOT of free memory (at least 4 gigs)
# using a swapfile of size 2 GB still did not fix my issues
# instead, I knew ahead of time the 0th tif is for Malawi, and the 1st tif is for Ethiopia and Nigeria
# I'll use this to only load one tif at a time
# thankfully, the countries did not span across two tifs
tif_array = np.squeeze(tifs[0].get_data())

In [ ]:
def add_nightlights(df, tif, tif_array):
    ''' 
    This takes a dataframe with columns cluster_lat, cluster_lon and finds the average 
    nightlights in 2015 using a 10kmx10km box around the point
    
    I try all the nighlights tifs until a match is found, or none are left upon which an error is raised
    '''
    cluster_nightlights = []
    for i,r in df.iterrows():
        min_lat, min_lon, max_lat, max_lon = create_space(r.cluster_lat, r.cluster_lon)
        
        xminPixel, ymaxPixel = tif.proj_to_raster(min_lon, min_lat)
        xmaxPixel, yminPixel = tif.proj_to_raster(max_lon, max_lat)
        assert xminPixel < xmaxPixel, print(r.cluster_lat, r.cluster_lon)
        assert yminPixel < ymaxPixel, print(r.cluster_lat, r.cluster_lon)
        if xminPixel < 0 or xmaxPixel >= tif_array.shape[1]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        elif yminPixel < 0 or ymaxPixel >= tif_array.shape[0]:
            print(f"no match for {r.cluster_lat}, {r.cluster_lon}")
            raise ValueError()
        xminPixel, yminPixel, xmaxPixel, ymaxPixel = int(xminPixel), int(yminPixel), int(xmaxPixel), int(ymaxPixel)
        cluster_nightlights.append(tif_array[yminPixel:ymaxPixel,xminPixel:xmaxPixel].mean())
        
    df['nightlights'] = cluster_nightlights

In [ ]:
add_nightlights(df_mw, tifs[0], tif_array)

In [ ]:
del tif_array
import gc
gc.collect()

5403

In [ ]:
import psutil
psutil.virtual_memory()

svmem(total=3883982848, available=3294400512, percent=15.2, used=440209408, free=3389562880, active=332414976, inactive=85729280, buffers=3567616, cached=50642944, shared=1142784, slab=24154112)

In [ ]:
tif_array = np.squeeze(tifs[1].get_data())

In [ ]:
add_nightlights(df_eth, tifs[1], tif_array)

In [ ]:
add_nightlights(df_ng, tifs[1], tif_array)

In [ ]:
df_mw.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,mw,-17.095150,35.217213,1.423239,0.025206
1,mw,-17.092351,35.114643,1.266204,0.000000
2,mw,-17.016698,35.079629,1.566870,0.000000
3,mw,-16.977243,35.205706,1.669245,0.008266
4,mw,-16.956385,35.168967,1.089891,0.002295


In [ ]:
df_eth.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,eth,3.455701,39.515994,14.854634,0.0
1,eth,3.549937,39.184234,14.312022,0.0
2,eth,3.864243,39.101366,12.470145,0.0
3,eth,3.982931,38.491368,8.336539,0.0
4,eth,4.048194,41.930928,4.762526,0.0


In [ ]:
df_ng.head()

,country,cluster_lat,cluster_lon,cons_pc,nightlights
0,ng,4.315786,6.268753,4.317717,0.123354
1,ng,4.328719,6.308172,4.880711,0.013713
2,ng,4.398427,7.183962,8.767258,38.470989
3,ng,4.425192,7.166935,10.774504,40.519035
4,ng,4.619377,7.684946,5.191333,0.000000


In [ ]:
df_mw['nightlights'].mean()

0.6038162894313969

In [ ]:
df_eth['nightlights'].mean()

0.6727544504689029

In [ ]:
df_ng['nightlights'].mean()

1.6584013095848769

In [ ]:
df_mw.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,-0.702793,-0.026563,-0.083273
cluster_lon,-0.702793,1.000000,-0.002947,-0.033367
cons_pc,-0.026563,-0.002947,1.000000,0.384939
nightlights,-0.083273,-0.033367,0.384939,1.000000


In [ ]:
df_eth.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.157130,-0.052673,-0.006627
cluster_lon,0.157130,1.000000,0.211846,0.076039
cons_pc,-0.052673,0.211846,1.000000,0.343879
nightlights,-0.006627,0.076039,0.343879,1.000000


In [ ]:
df_ng.corr()

,cluster_lat,cluster_lon,cons_pc,nightlights
cluster_lat,1.000000,0.330910,-0.376356,-0.188281
cluster_lon,0.330910,1.000000,-0.246189,-0.196671
cons_pc,-0.376356,-0.246189,1.000000,0.264398
nightlights,-0.188281,-0.196671,0.264398,1.000000


In [ ]:
for country in ['malawi_2016', 'ethiopia_2015', 'nigeria_2015']:
    os.makedirs(os.path.join(COUNTRIES_DIR, country, 'processed'), exist_ok=True)

In [ ]:
df_mw.to_csv(os.path.join(COUNTRIES_DIR, 'malawi_2016', 'processed/clusters.csv'), index=False)

In [ ]:
df_eth.to_csv(os.path.join(COUNTRIES_DIR, 'ethiopia_2015', 'processed/clusters.csv'), index=False)

In [ ]:
df_ng.to_csv(os.path.join(COUNTRIES_DIR, 'nigeria_2015', 'processed/clusters.csv'), index=False)